# Data Collection, Cleaning, and Pre-processing


The following code demonstrates our steps in collecting, cleaning, and pre-processing our data.

## Data Collection - EPA Water Quality Data

The dataset examined here was obtained from the US Environmental Protection Agency (EPA): https://www.epa.gov/dwucmr/occurrence-data-unregulated-contaminant-monitoring-rule#4. There are multiple files that contain water quality data points for PFAS contamination over the years throughout the United States. Only UCMR 5, which contains PFAS data for 2023-2025, and UCMR 3, which contains PFAS data from 2013 to 2015 were used. UCMR 4 contained contaminant data for various types of carcinogens, heavy metals, etc. but not PFAS. UCMR, or the Unregulated Contaminant Monitoring Rule, is how the EPA collects data for contaminants that might be in drinking water but do not have regulatory standards under the Safe Drinking Water Act.

The data was collected through a script that outputs the zip file from a given website. This dataset is relevant to our research questions since it contains the PFAS levels that were sampled in our water sources across the country. We are interested in determining how PFAS contamination varies with other parameters such as geography, time, poverty, state boundaries, water sources, etc. This will allow for us to further analyze contamination patterns and identify at-risk areas and populations.

In [2]:
# Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
import matplotlib.patches as patches
import geopandas as gpd
from scipy import stats
import seaborn as sns
import re
from IPython.display import display
import zipfile
import io

### Collecting the data from EPA's website

In [3]:
# Target directory
directory ='C:/Users/chenc/Downloads/ucmr_data'

def get_epa_data(url, directory):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        print('Download started...')

        # Open the zip file from the response content
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            # Extract all the contents of the zip file into the custom directory
            zip_ref.extractall(directory)

        print(f'Download and extraction completed. Files saved in {directory}')
    else:
        print(f'Failed to download data. HTTP status code: {response.status_code}')
    print(os.getcwd())

# URL for the ZIP file
urls = ['https://www.epa.gov/system/files/other-files/2023-08/ucmr5-occurrence-data.zip',
        'https://www.epa.gov/system/files/other-files/2024-04/ucmr3-occurrence-data.zip']


for link in urls:
    get_epa_data(link, directory)

Download started...
Download and extraction completed. Files saved in C:/Users/chenc/Downloads/ucmr_data
/Users/moritzknodler/Documents/00_Lectures/0_Data Mining/Project
Download started...
Download and extraction completed. Files saved in C:/Users/chenc/Downloads/ucmr_data
/Users/moritzknodler/Documents/00_Lectures/0_Data Mining/Project


The datasets are then uploaded to Github where all collaborators can pull from to avoid having to use local storage.

### Data Cleaning and Preprocessing

The raw datasets are read into a dataframe and combined.

In [4]:
#Read in data from first file
ucmr3=pd.read_csv("https://github.com/chencheny249/DataMiningProjSpring2025/blob/5777a5bb0d8dd7eaa06f48730eb5c3c39cf282ae/UCMR3_All.txt?raw=true", encoding='ISO-8859-1', delimiter='\t')
ucmr3.head()

,PWSID,PWSName,Size,FacilityID,FacilityName,FacilityWaterType,SamplePointID,SamplePointName,SamplePointType,AssociatedFacilityID,...,MRL,Units,MethodID,AnalyticalResultsSign,AnalyticalResultValue,SampleEventCode,MonitoringRequirement,Region,State,UCMR1SampleType
0,010106001,Mashantucket Pequot Water System,L,6,MPTN WTP,GU,D11,WTP EPTDS,EP,202.0,...,0.07,µg/L,EPA 522,<,NaN,SE1,AM,1,01,NaN
1,010106001,Mashantucket Pequot Water System,L,6,MPTN WTP,GU,D11,WTP EPTDS,EP,202.0,...,20.00,µg/L,EPA 300.1,=,160.0,SE1,AM,1,01,NaN
2,010106001,Mashantucket Pequot Water System,L,6,MPTN WTP,GU,D11,WTP EPTDS,EP,202.0,...,0.20,µg/L,EPA 200.8,<,NaN,SE1,AM,1,01,NaN
3,010106001,Mashantucket Pequot Water System,L,6,MPTN WTP,GU,D11,WTP EPTDS,EP,202.0,...,0.30,µg/L,EPA 200.8,=,72.0,SE1,AM,1,01,NaN
4,010106001,Mashantucket Pequot Water System,L,6,MPTN WTP,GU,D11,WTP EPTDS,EP,202.0,...,1.00,µg/L,EPA 200.8,<,NaN,SE1,AM,1,01,NaN


In [5]:
#Read in data from last file
ucmr5=pd.read_csv("https://github.com/chencheny249/DataMiningProjSpring2025/blob/5777a5bb0d8dd7eaa06f48730eb5c3c39cf282ae/UCMR5_All.txt?raw=true", encoding='ISO-8859-1', delimiter='\t')
ucmr5.head()

,PWSID,PWSName,Size,FacilityID,FacilityName,FacilityWaterType,SamplePointID,SamplePointName,SamplePointType,AssociatedFacilityID,...,MRL,Units,MethodID,AnalyticalResultsSign,AnalyticalResultValue,SampleEventCode,MonitoringRequirement,Region,State,UCMR1SampleType
0,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.002,µg/L,EPA 533,<,NaN,SE1,AM,1,01,NaN
1,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.005,µg/L,EPA 537.1,<,NaN,SE1,AM,1,01,NaN
2,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.004,µg/L,EPA 533,<,NaN,SE1,AM,1,01,NaN
3,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.003,µg/L,EPA 533,=,0.0035,SE1,AM,1,01,NaN
4,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.007,µg/L,EPA 537.1,<,NaN,SE1,AM,1,01,NaN


In [6]:
#merge datasets
comb_df=pd.concat([ucmr5, ucmr3], ignore_index=True, sort=False)
comb_df.head()

,PWSID,PWSName,Size,FacilityID,FacilityName,FacilityWaterType,SamplePointID,SamplePointName,SamplePointType,AssociatedFacilityID,...,MRL,Units,MethodID,AnalyticalResultsSign,AnalyticalResultValue,SampleEventCode,MonitoringRequirement,Region,State,UCMR1SampleType
0,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.002,µg/L,EPA 533,<,NaN,SE1,AM,1,01,NaN
1,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.005,µg/L,EPA 537.1,<,NaN,SE1,AM,1,01,NaN
2,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.004,µg/L,EPA 533,<,NaN,SE1,AM,1,01,NaN
3,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.003,µg/L,EPA 533,=,0.0035,SE1,AM,1,01,NaN
4,010106001,Mashantucket Pequot Water System,L,00006,MPTN WTP,GU,TP1,Entry point to Dist. System,EP,NaN,...,0.007,µg/L,EPA 537.1,<,NaN,SE1,AM,1,01,NaN


In [7]:
#Check if merge worked
ucmr5.shape[0]+ucmr3.shape[0]==comb_df.shape[0]

True

We checked for any NA's. There are a quite a few in certain columns. How we handled the NAs are described below.

In [8]:
#Check for NAs
comb_df.isna().sum()

PWSID                            0
PWSName                          0
Size                             0
FacilityID                      60
FacilityName                    30
FacilityWaterType                0
SamplePointID                    0
SamplePointName                  0
SamplePointType                  0
AssociatedFacilityID       1570414
AssociatedSamplePointID    1570414
CollectionDate                   0
SampleID                         0
Contaminant                      0
MRL                              0
Units                            0
MethodID                         0
AnalyticalResultsSign            0
AnalyticalResultValue      2138363
SampleEventCode                  0
MonitoringRequirement            0
Region                           0
State                            0
UCMR1SampleType            2430841
dtype: int64

FacilityID and FacilityName: since they refer to the same thing, any FacilityName with a NA will be filled with its corresponding FacilityID, if there is one.

AssociatedFacilityID and AssociatedSamplePointID: these are all null per the dataset technical documents. These columns will be deleted.

MRL: Certain contaminants do not have a MRL (minimum reporting level). This value does not have any health implications - it is just the lowest value that labs can report. NAs will be replaced with -1. It is the Anlaytical Result Value that tells us the actual concentration.

AnalyticalResultValue: per the technical documentation provided by the EPA, this value is NA when it is under the MRL. For all intents and purposes, these NAs will be replaced with 0 since they are lower than the minimum value labs need to report.

UCMR1SampleType: This is not needed for our analysis and will be deleted.

In [9]:
#Fill NAs as described above
comb_df.fillna({'AnalyticalResultValue':0}, inplace=True)
comb_df.fillna({'FacilityName':comb_df['FacilityID']}, inplace=True)
comb_df.fillna({'MRL':-1}, inplace=True)

Since there are so many columns, I will be removing a bunch that are unnecessary or redundant. PWSName and PWSID are redundant since they refer to the same PWS, just either with a number or the actual name. In case we need to look it up later, a dictionary connecting the names and the corresponding IDs were made, and the PWSID column was dropped. The other dropped columns did not aid our investigation, hence why they were removed.

In [10]:
#Create dictionary for PWS Names and IDs (ID column will be deleted, the dictionary allows us to call it back by name)
PWS_dict=dict(zip(comb_df['PWSName'],comb_df['PWSID']))
PWS_dict['Mashantucket Pequot Water System']

'010106001'

In [11]:
#Drop unnecessary columns
columns_to_drop = ['PWSID', 'FacilityID', 'SamplePointID', "MethodID", "AssociatedFacilityID",
                   "AssociatedSamplePointID", "SampleEventCode", "UCMR1SampleType",
                   'SamplePointType', 'Units', 'MonitoringRequirement']

comb_df = comb_df.drop(columns=[col for col in columns_to_drop if col in comb_df.columns])

# Filter out non-state codes
fake_states = ["DC", "PR", "GU", "NN", "VI", "MP", "AS", "FM", "MH", "PW"]
comb_df = comb_df[~comb_df["State"].isin(fake_states)]

comb_df = comb_df[~comb_df["State"].astype(str).str.contains("0")]  # Remove states with "0"

# Verify filtering
print("Unique states remaining:",comb_df["State"].nunique())
print(comb_df["State"].unique())  # Check remaining states

print(comb_df.shape)  # Check number of rows after filtering


comb_df.head()

Unique states remaining: 50
['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND'
 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN'
 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']
(2369915, 13)


,PWSName,Size,FacilityName,FacilityWaterType,SamplePointName,CollectionDate,SampleID,Contaminant,MRL,AnalyticalResultsSign,AnalyticalResultValue,Region,State
6815,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,11/4/2024,32403661003,PFNA,0.004,<,0.0,10,AK
6816,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,11/4/2024,32403661003,PFUnA,0.002,<,0.0,10,AK
6817,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,11/4/2024,32403661003,NEtFOSAA,0.005,<,0.0,10,AK
6818,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,11/4/2024,32403661003,PFHpS,0.003,<,0.0,10,AK
6819,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,11/4/2024,32403661003,9Cl-PF3ONS,0.002,<,0.0,10,AK


In [12]:
#Check for NAs - None!
comb_df.isna().sum()

PWSName                  0
Size                     0
FacilityName             0
FacilityWaterType        0
SamplePointName          0
CollectionDate           0
SampleID                 0
Contaminant              0
MRL                      0
AnalyticalResultsSign    0
AnalyticalResultValue    0
Region                   0
State                    0
dtype: int64

Here is an overview of the data.

In [13]:
"""Data overview"""

# Display dataset overview
print("\n📌 Dataset Shape (Rows, Columns):")
print(comb_df.shape)

print("\n📌 Column Names:")
print(comb_df.columns.tolist())

print("\n📌 General Info:")
comb_df.info()  # No need to use print() since info() already prints output

print("\n📌 Missing Values Count per Column:")
print(comb_df.isna().sum())

print("\n📌 Percentage of Missing Values per Column:")
print(comb_df.isna().mean() * 100)

print("\n📌 Unique Value Count per Column:")
print(comb_df.nunique())

print("\n📌 Summary Statistics for Categorical Columns:")
print(comb_df.describe(include="object"))

print("\n📌 Number of Duplicate Rows:")
print(comb_df.duplicated().sum())

print("\n📌 Columns with Only One Unique Value:")
print(comb_df.nunique()[comb_df.nunique() == 1])


📌 Dataset Shape (Rows, Columns):
(2369915, 13)

📌 Column Names:
['PWSName', 'Size', 'FacilityName', 'FacilityWaterType', 'SamplePointName', 'CollectionDate', 'SampleID', 'Contaminant', 'MRL', 'AnalyticalResultsSign', 'AnalyticalResultValue', 'Region', 'State']

📌 General Info:
<class 'pandas.core.frame.DataFrame'>
Index: 2369915 entries, 6815 to 2430840
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   PWSName                object 
 1   Size                   object 
 2   FacilityName           object 
 3   FacilityWaterType      object 
 4   SamplePointName        object 
 5   CollectionDate         object 
 6   SampleID               object 
 7   Contaminant            object 
 8   MRL                    float64
 9   AnalyticalResultsSign  object 
 10  AnalyticalResultValue  float64
 11  Region                 int64  
 12  State                  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 253.1+ MB

📌 Mi

In [14]:
"""Data overview 2: list of unique values per column"""

for col in comb_df.columns:
    unique_values = comb_df[col].unique()  # Get unique values
    print(f"\n📌 Unique values in column: {col} ({len(unique_values)} unique values)")
    print(unique_values[:10])  # Show only first 10 unique values for readability


📌 Unique values in column: PWSName (13458 unique values)
['CITY AND BOROUGH OF JUNEAU' 'VALLI VUE S/D WATER SYSTEM'
 'MOA MUNICIPALITY OF ANCHORAGE' 'JBER-ELMENDORF'
 'DOYON UTILITIES JBER - RICHARDSON' 'WASILLA WATER SYSTEM'
 'PALMER WATER SYSTEM' 'KENAI WATER SYSTEM' 'SEWARD' 'SOLDOTNA']

📌 Unique values in column: Size (2 unique values)
['L' 'S']

📌 Unique values in column: FacilityName (20104 unique values)
['Salmon Creek Reservoir TP' 'Last Chance Basin Wells TP' 'Valli Vue'
 'Wells #26 & #27 TP' 'Eklutna WTF' 'Ship Creek TP' 'Well # 12 TP'
 'Well #4 TP' 'Well #7 TP' 'Well #9 TP']

📌 Unique values in column: FacilityWaterType (4 unique values)
['SW' 'GW' 'MX' 'GU']

📌 Unique values in column: SamplePointName (28196 unique values)
['Salmon Creek DS EP' 'Last Chance Basin DS EP'
 'Valli Vue Wellhouse Exit to Distribution' 'Wells #26 & #27 DS EP'
 'Eklutna DS EP' 'Ship Creek DS EP' 'Well #12 DS EP' 'Well #4 DS EP'
 'Well #7 DS EP' 'Well #9 DS EP']

📌 Unique values in column: Collect

Since the data contains more than just PFAS - there are heavy metals and other carcinogens, we filtered it for just PFAS values. The other contaminants are important, but beyond the scope of this study. Since we want to know how PFAS contamination behaves over time, we converted the CollectionDate to a datetime format and added columns for Year and Month for easier retrieval and analysis later.

We also calculated which samples had a measured value over or under the MRL, or minimum required level. This is the minimum level labs are required to report to the EPA. While the MRL does not have any health indications, knowing that some contaminants are not at the MRL while some are over is still useful. The number of values that are above or at/below the MRL are counted, and for those that are above, the relative contamination level is calculated by taking the recorded value divided by the MRL.

In [15]:
"""Data preprocessing 1: filter for PFAS, add cols for Year and Month, calculate relative contamination levels"""


# Filter data for PFAS contaminants
pfas_list = [
    "PFOA", "PFBS", "PFHpA", "PFHxS", "PFNA", "PFOS", "9Cl-PF3ONS", "NEtFOSAA",
    "PFTrDA", "ADONA", "PFTA", "NMeFOSAA", "PFHxA", "HFPO-DA", "PFPeS", "PFHpS",
    "PFDoA", "PFMPA", "11Cl-PF3OUdS", "PFDA", "PFEESA", "PFUnA", "PFBA", "PFPeA",
    "8:2 FTS", "NFDHA", "6:2 FTS", "4:2 FTS", "PFMBA"
]

df_PFAS = comb_df[comb_df["Contaminant"].isin(pfas_list)].copy()


# Convert 'CollectionDate' to datetime, drop rows with missing datetime string
# Add cols for Year and Month
df_PFAS["CollectionDate"] = pd.to_datetime(df_PFAS["CollectionDate"], errors='coerce')
df_PFAS = df_PFAS.dropna(subset=["CollectionDate"])

df_PFAS["Year"] = df_PFAS["CollectionDate"].dt.year
df_PFAS["Month"] = df_PFAS["CollectionDate"].dt.month


# Add cols for MRL level exceeded: Count '<' and '=' occurrences and create seperate columns, add a col for count of both (total)
df_PFAS.loc[:, 'Count_Less_Than'] = (df_PFAS['AnalyticalResultsSign'] == '<').astype(int)
df_PFAS.loc[:, 'Count_Equals'] = (df_PFAS['AnalyticalResultsSign'] == '=').astype(int)


# Add col for Relative Contamination level (delta AnalyticalResultValue / MRL); only for values with AnalyticalResultsSign "="
# This divides the measured value for samples exceeding the threshold MRL, by the MRL value
# Result is a measure how much the MRL is exceeded
df_PFAS.loc[df_PFAS["AnalyticalResultsSign"] == "=", "Relative_Contamination_Level"] = (
    df_PFAS["AnalyticalResultValue"] / df_PFAS["MRL"]
)


print("\n📌 Processed data -- General Info:")
df_PFAS.info()

display(df_PFAS)



📌 Processed data -- General Info:
<class 'pandas.core.frame.DataFrame'>
Index: 1499860 entries, 6815 to 2430840
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   PWSName                       1499860 non-null  object        
 1   Size                          1499860 non-null  object        
 2   FacilityName                  1499860 non-null  object        
 3   FacilityWaterType             1499860 non-null  object        
 4   SamplePointName               1499860 non-null  object        
 5   CollectionDate                1499860 non-null  datetime64[ns]
 6   SampleID                      1499860 non-null  object        
 7   Contaminant                   1499860 non-null  object        
 8   MRL                           1499860 non-null  float64       
 9   AnalyticalResultsSign         1499860 non-null  object        
 10  AnalyticalResultValue         149

,PWSName,Size,FacilityName,FacilityWaterType,SamplePointName,CollectionDate,SampleID,Contaminant,MRL,AnalyticalResultsSign,AnalyticalResultValue,Region,State,Year,Month,Count_Less_Than,Count_Equals,Relative_Contamination_Level
6815,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,PFNA,0.004,<,0.0,10,AK,2024,11,1,0,NaN
6816,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,PFUnA,0.002,<,0.0,10,AK,2024,11,1,0,NaN
6817,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,NEtFOSAA,0.005,<,0.0,10,AK,2024,11,1,0,NaN
6818,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,PFHpS,0.003,<,0.0,10,AK,2024,11,1,0,NaN
6819,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,9Cl-PF3ONS,0.002,<,0.0,10,AK,2024,11,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430825,Casper Board of Public Utilities,L,GUDI Plant,GU,EPTDS from GUDI Plant,2015-12-16,201511140097AM,PFBS,0.090,<,0.0,8,WY,2015,12,1,0,NaN
2430826,Casper Board of Public Utilities,L,GUDI Plant,GU,EPTDS from GUDI Plant,2015-12-16,201511140097AM,PFHpA,0.010,<,0.0,8,WY,2015,12,1,0,NaN
2430827,Casper Board of Public Utilities,L,GUDI Plant,GU,EPTDS from GUDI Plant,2015-12-16,201511140097AM,PFHxS,0.030,<,0.0,8,WY,2015,12,1,0,NaN
2430828,Casper Board of Public Utilities,L,GUDI Plant,GU,EPTDS from GUDI Plant,2015-12-16,201511140097AM,PFNA,0.020,<,0.0,8,WY,2015,12,1,0,NaN


## Data Collection - US Poverty Data
The dataset examined here was obtained from https://www.census.gov/data/developers/data-sets/Poverty-Statistics.html, and provides small area income and poverty esitmates (SAIPE) of income and poverty statistics from states. The data aims to provide estimates of income and poverty for the administration of federal programs.

The data was collected through an API call from census.gov. The code for the API implementation is given below. This dataset is relevant to our research questions since we are interested in determining if PFAs contamination have any correlation with poverty levels, as well as region. It also provides insight into population/density features and their relationship with poverty levels, which may also provide further details on the risk of PFAs.

In [16]:
#API REQUEST TO GET POVERTY DATA

url = "https://api.census.gov/data/timeseries/poverty/saipe"
params = {
    #SPECFIC VARIABLES OF INTEREST - Ensuring Project Relevance
    # Added County Column
    "get": "NAME,COUNTY,SAEMHI_PT,SAEPOV0_17_PT,SAEPOVRT0_17_PT,SAEPOVALL_PT,SAEPOVRTALL_PT,SAEPOVU_0_17,SAEPOVU_ALL,STABREV,STATE,YEAR",
    "for": "county:*",
    "in": "state:*"
}

response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])

else:
    print(f"Request failed with status code {response.status_code}")

# CREATE DATA FRAME
df = pd.DataFrame(data[1:], columns=data[0])

### Data Cleaning and Preprocessing

### Understanding the Data

The following code cleans this dataset to make it more suitable for further analysis. Missing values, duplicates, or any outliers are assessed along with updating the dataset to include more interpretable column names and only include relevant years in relation to the other dataset used (EPA data). It also assess the data for completeness, consistency, and usability.

In [17]:
df.head()

,NAME,COUNTY,SAEMHI_PT,SAEPOV0_17_PT,SAEPOVRT0_17_PT,SAEPOVALL_PT,SAEPOVRTALL_PT,SAEPOVU_0_17,SAEPOVU_ALL,STABREV,STATE,YEAR,state,county
0,Autauga County,001,26898,2071,20.2,4956,14.3,None,None,AL,01,1989,01,001
1,Baldwin County,003,24043,4838,18.6,13031,13.2,None,None,AL,01,1989,01,003
2,Barbour County,005,18673,2826,38.2,6601,26.2,None,None,AL,01,1989,01,005
3,Bibb County,007,19604,1206,25.0,3133,18.9,None,None,AL,01,1989,01,007
4,Blount County,009,24035,1972,19.4,5296,13.4,None,None,AL,01,1989,01,009


In [18]:
# Making Column Names more interpretable
df = df.rename(columns={'SAEMHI_PT': 'Median_Income',
                        'SAEPOV0_17_PT': 'Child_Poverty_Count',
                        'SAEPOVRT0_17_PT': 'Child_Poverty_Rate',
                        'SAEPOVALL_PT': 'Poverty_Count',
                        'SAEPOVRTALL_PT': 'Poverty_Rate',
                        'SAEPOVU_0_17': 'All_Child_Poverty_Count',
                        'SAEPOVU_ALL': 'All_Poverty_Count'})

# Drop redundant Columns
df = df.drop(columns = [ 'state' ])


# Convert Columns to proper data type
exclude_columns = ['NAME', 'STABREV']
columns_to_convert = [col for col in df.columns if col not in exclude_columns]
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce', downcast='integer')

# Filter for relevant years
years_of_interest = [2013, 2014, 2015, 2018, 2019, 2020, 2023]
df = df[df['YEAR'].isin(years_of_interest)]

# Examine Data Types
df.info()

# Check first few rows (adding County as name)
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 21998 entries, 62832 to 97400
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NAME                     21998 non-null  object 
 1   COUNTY                   21998 non-null  int16  
 2   Median_Income            21991 non-null  float64
 3   Child_Poverty_Count      21991 non-null  float64
 4   Child_Poverty_Rate       21991 non-null  float64
 5   Poverty_Count            21991 non-null  float64
 6   Poverty_Rate             21991 non-null  float64
 7   All_Child_Poverty_Count  21991 non-null  float64
 8   All_Poverty_Count        21991 non-null  float64
 9   STABREV                  21998 non-null  object 
 10  STATE                    21998 non-null  int8   
 11  YEAR                     21998 non-null  int16  
 12  county                   21998 non-null  int16  
dtypes: float64(7), int16(3), int8(1), object(2)
memory usage: 1.8+ MB


,NAME,COUNTY,Median_Income,Child_Poverty_Count,Child_Poverty_Rate,Poverty_Count,Poverty_Rate,All_Child_Poverty_Count,All_Poverty_Count,STABREV,STATE,YEAR,county
62832,Autauga County,1,51868.0,2724.0,19.6,7397.0,13.5,13903.0,54762.0,AL,1,2013,1
62833,Baldwin County,3,47539.0,8994.0,20.8,27429.0,14.2,43240.0,192931.0,AL,1,2013,3
62834,Barbour County,5,30981.0,2393.0,42.4,6654.0,28.2,5644.0,23617.0,AL,1,2013,5
62835,Bibb County,7,39781.0,1664.0,35.6,4689.0,23.1,4679.0,20266.0,AL,1,2013,7
62836,Blount County,9,44392.0,3637.0,26.6,9878.0,17.2,13648.0,57315.0,AL,1,2013,9


In [19]:
missing_rows = df[df.isna().any(axis=1)]

# View rows with missing data
missing_rows.head()

,NAME,COUNTY,Median_Income,Child_Poverty_Count,Child_Poverty_Rate,Poverty_Count,Poverty_Rate,All_Child_Poverty_Count,All_Poverty_Count,STABREV,STATE,YEAR,county
63380,Kalawao County,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HI,15,2013,5
66523,Kalawao County,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HI,15,2014,5
69665,Kalawao County,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HI,15,2015,5
79091,Kalawao County,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HI,15,2018,5
82233,Kalawao County,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HI,15,2019,5


In [20]:
#Exaimine NA Values
# Fill missing values for a specific column with the median (less susceptible to outliers)
df['Median_Income'] = df['Median_Income'].fillna(df['Median_Income'].median())
df['Child_Poverty_Count'] = df['Child_Poverty_Count'].fillna(df['Child_Poverty_Count'].median())
df['Child_Poverty_Rate'] = df['Child_Poverty_Rate'].fillna(df['Child_Poverty_Rate'].median())
df['Poverty_Count'] = df['Poverty_Count'].fillna(df['Poverty_Count'].median())
df['Poverty_Rate'] = df['Poverty_Rate'].fillna(df['Poverty_Rate'].median())
df['All_Child_Poverty_Count'] = df['All_Child_Poverty_Count'].fillna(df['All_Child_Poverty_Count'].median())
df['All_Poverty_Count'] = df['All_Poverty_Count'].fillna(df['All_Poverty_Count'].median())

# Check if there are still any missing values
df.isna().sum().sum()

np.int64(0)

## Collecting PWS County Names with Google API

The script for collecting county names that contain each PWS is located at https://github.com/chencheny249/DataMiningProjSpring2025/blob/fd09212dda9740ab291478d5caf9814a723543a7/20250319_Get_county_from_api.ipynb. We scraped this data through Google Maps's API so that we can get a more detailed, county-level view of the PWS's and contamination patterns across the country. The results were saved in a csv file, and the new dataframe will be combined with the rest of the water quality and poverty data. 

In [21]:
pws_counties=pd.read_csv("https://github.com/chencheny249/DataMiningProjSpring2025/blob/fd09212dda9740ab291478d5caf9814a723543a7/pws_with_counties.csv?raw=true")
pws_counties.head()

,State,PWSName,County
0,PR,GURABO URBANO,Rincón
1,GA,CUMMING,Forsyth County
2,IA,ASBURY MUNICIPAL WATER SYSTEM,Dubuque County
3,OK,MUSKOGEE,Muskogee County
4,TX,Lake Cities Municipal Utility Authority,Denton County


## Combining datasets

We want the corresponding poverty data for each water sample entry, which is by location and time, or State and Year (poverty data is on an annual basis). After some more cleaning, the abbreviated state column and the year column was renamed to match, and the two dataframes were merged.

Originally, any contaminant with a measured level at or below the MRL had 'NA' as their relative contamination level. A decision was then made to fill any of those NAs with zeros - since the levels were so low that they functionally zero. There were also some redundant columns - NAME (full state name) and STATE (numbers of states) were removed.

After merging, some samples show NAs for their poverty data. The samples from those areas are technically on tribal land, and therefore do not belong to a state - they are designated by their EPA regions instead. Any state analysis will therefore not include tribal data, but analysis based on EPA region will.

The merged dataframe was then split into 12 smaller dataframes for uploading to Github due to file size restrictions.

In [22]:
#Look at the poverty data
df_poverty = df
df_poverty.head()

,NAME,COUNTY,Median_Income,Child_Poverty_Count,Child_Poverty_Rate,Poverty_Count,Poverty_Rate,All_Child_Poverty_Count,All_Poverty_Count,STABREV,STATE,YEAR,county
62832,Autauga County,1,51868.0,2724.0,19.6,7397.0,13.5,13903.0,54762.0,AL,1,2013,1
62833,Baldwin County,3,47539.0,8994.0,20.8,27429.0,14.2,43240.0,192931.0,AL,1,2013,3
62834,Barbour County,5,30981.0,2393.0,42.4,6654.0,28.2,5644.0,23617.0,AL,1,2013,5
62835,Bibb County,7,39781.0,1664.0,35.6,4689.0,23.1,4679.0,20266.0,AL,1,2013,7
62836,Blount County,9,44392.0,3637.0,26.6,9878.0,17.2,13648.0,57315.0,AL,1,2013,9


In [23]:
# Look at the PFAS data
df_PFAS.head()

,PWSName,Size,FacilityName,FacilityWaterType,SamplePointName,CollectionDate,SampleID,Contaminant,MRL,AnalyticalResultsSign,AnalyticalResultValue,Region,State,Year,Month,Count_Less_Than,Count_Equals,Relative_Contamination_Level
6815,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,PFNA,0.004,<,0.0,10,AK,2024,11,1,0,NaN
6816,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,PFUnA,0.002,<,0.0,10,AK,2024,11,1,0,NaN
6817,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,NEtFOSAA,0.005,<,0.0,10,AK,2024,11,1,0,NaN
6818,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,PFHpS,0.003,<,0.0,10,AK,2024,11,1,0,NaN
6819,CITY AND BOROUGH OF JUNEAU,L,Salmon Creek Reservoir TP,SW,Salmon Creek DS EP,2024-11-04,32403661003,9Cl-PF3ONS,0.002,<,0.0,10,AK,2024,11,1,0,NaN


In [24]:
#Fill Relative Contamination Level NA's with zeros
df_PFAS.fillna({'Relative_Contamination_Level':0}, inplace=True)

In [25]:
# Rename columns so that they match across dataframes
df_poverty=df_poverty.rename(columns={'STABREV':'State','YEAR':'Year'})

df_poverty.rename(columns={"NAME": "County", "COUNTY": "County_ID","STATE":"State_ID"}, inplace=True)

# Drop the unnecessary columns
df_poverty.drop(columns=['Poverty_Count','All_Child_Poverty_Count',"All_Poverty_Count",'Child_Poverty_Count',"county", "County_ID", "State_ID"], errors="ignore", inplace=True)

# Reorder columns: Put 'State' first, keep all other columns as they are
cols = ["State"] + [col for col in df_poverty.columns if col != "State"]
df_poverty = df_poverty[cols]

# Verify changes
display(df_poverty.sample(5))


,State,County,Median_Income,Child_Poverty_Rate,Poverty_Rate,Year
87269,TN,Clay County,37683.0,30.1,21.0,2020
79025,GA,Macon County,32868.0,39.2,30.5,2018
87239,SD,Moody County,73264.0,10.6,9.4,2020
94795,GA,Washington County,48809.0,29.5,21.6,2023
85878,KY,Johnson County,40455.0,28.0,22.5,2020


In [26]:
pws_counties.head()

,State,PWSName,County
0,PR,GURABO URBANO,Rincón
1,GA,CUMMING,Forsyth County
2,IA,ASBURY MUNICIPAL WATER SYSTEM,Dubuque County
3,OK,MUSKOGEE,Muskogee County
4,TX,Lake Cities Municipal Utility Authority,Denton County


In [27]:
# MERGE 1: Add counties matched by PWSName to the main data frame df_PFAS

# Remove any numbers and parantheses from PWSName column
import re
# Function to clean PWSName (removes numbers & parentheses)
def clean_pws_name(name):
    return re.sub(r"\s*\([^)]*\)|\d+", "", name).strip()

# Apply function to the PWSName column
df_PFAS["PWSName"] = df_PFAS["PWSName"].apply(clean_pws_name)



In [28]:
# MERGE 1

df_PFAS = df_PFAS.merge(pws_counties, on=["State", "PWSName"], how="left")

# If merge created 'County_x' and 'County_y', combine them
if "County_x" in df_PFAS.columns and "County_y" in df_PFAS.columns:
    df_PFAS["County"] = df_PFAS["County_y"].fillna(df_PFAS["County_x"])
    df_PFAS.drop(columns=["County_x", "County_y"], inplace=True)
elif "County" in df_PFAS.columns:
    print("Merge did not create duplicate columns, using existing 'County'.")

# Move 'County' column to the front
cols = ["County"] + [col for col in df_PFAS.columns if col != "County"]
df_PFAS = df_PFAS[cols]


Merge did not create duplicate columns, using existing 'County'.


In [29]:
# MERGE 1
# # Verify merge
display(df_PFAS.sample(10))

,County,PWSName,Size,FacilityName,FacilityWaterType,SamplePointName,CollectionDate,SampleID,Contaminant,MRL,AnalyticalResultsSign,AnalyticalResultValue,Region,State,Year,Month,Count_Less_Than,Count_Equals,Relative_Contamination_Level
975786,Chesterfield County,CHERAW TOWN OF,S,TOWN OF CHERAW,SW,FINISHED ENTRY POINT,2024-06-19,117454P,ADONA,0.003,<,0.0000,4,SC,2024,6,1,0,0.0
1095862,Harrison County,CITY OF HALLSVILLE,S,Longview Intertie,SW,Hwy 80 Pumpstation,2024-04-15,116003P,9Cl-PF3ONS,0.002,<,0.0000,6,TX,2024,4,1,0,0.0
1381260,St. Tammany Parish,St. Tammany Parish - Briarwood Terrace,L,Briarwood Terrace Well,GW,EPTDS from Briarwood Terrace Well,2014-07-28,35148033001AM,PFOS,0.040,<,0.0000,6,LA,2014,7,1,0,0.0
152911,Merced County,CITY OF ATWATER,L,Well 19,GW,EP #30 - Well 19 Treated,2023-10-31,AGK0166-03,6:2 FTS,0.005,<,0.0000,9,CA,2023,10,1,0,0.0
1465813,Cameron County,East Rio Hondo WSC,L,Meter from N. Cameron,GW,Sample Tap,2013-09-30,Q1308427009,PFOA,0.020,<,0.0000,6,TX,2013,9,1,0,0.0
1469145,Galveston County,City of Galveston,L,Gulf Coast Water Intertie,SW,EPTDS from GCWA TX City,2015-12-21,Q1550161001,PFNA,0.020,<,0.0000,6,TX,2015,12,1,0,0.0
1004744,Roane County,KINGSTON WATER SYSTEM,S,Pump Station,GW,pump station,2024-07-16,119069P,PFPeS,0.004,<,0.0000,4,TN,2024,7,1,0,0.0
145658,Los Angeles County,"BURBANK-CITY, WATER DEPT.",L,SWTP,SW,EP #53 - Blended Effluent,2024-08-05,380-106931-1,PFHxS,0.003,<,0.0000,9,CA,2024,8,1,0,0.0
1471732,Harris County,City of Seabrook,L,Pasadena Water Intertie,SW,EPTDS from Pasadena,2015-04-15,Q1514047001,PFOA,0.020,<,0.0000,6,TX,2015,4,1,0,0.0
447458,Will County,CREST HILL,L,TP 8 (Well 10),GW,TP 8 Finished Water,2023-12-28,810-89247-1,PFBS,0.003,=,0.0285,5,IL,2023,12,0,1,9.5


In [30]:
# MERGE 2: Add poverty data by county

df_PFAS = df_PFAS.merge(df_poverty, on=["State", "County", "Year"], how="left")


df_PFAS = df_PFAS.merge(df_poverty, on=["State", "County", "Year"], how="left")

# If merge created 'County_x' and 'County_y', combine them
if "County_x" in df_PFAS.columns and "County_y" in df_PFAS.columns:
    df_PFAS["County"] = df_PFAS["County_x"].fillna(df_PFAS["County_y"])
    df_PFAS.drop(columns=["County_x", "County_y"], inplace=True)


# Loop through columns and replace duplicates
for col in df_PFAS.columns:
    if col.endswith("_y") and col.replace("_y", "_x") in df_PFAS.columns:
        df_PFAS[col.replace("_y", "")] = df_PFAS[col].fillna(df_PFAS[col.replace("_y", "_x")])  # Keep values
        df_PFAS.drop(columns=[col, col.replace("_y", "_x")], inplace=True)  # Drop both _x and _y


# Verify the cleaned DataFrame
print(df_PFAS.shape)

display(df_PFAS.sample(5))


(1499860, 22)


,County,PWSName,Size,FacilityName,FacilityWaterType,SamplePointName,CollectionDate,SampleID,Contaminant,MRL,...,Region,State,Year,Month,Count_Less_Than,Count_Equals,Relative_Contamination_Level,Median_Income,Child_Poverty_Rate,Poverty_Rate
801701,Clark County,COSMOPOLITAN HOTEL,S,Cosmopolitan,SW,The Henry Restaurant Faucet Port,2024-01-23,112175P,PFHpA,0.003,...,9,NV,2024,1,1,0,0.0,NaN,NaN,NaN
1367033,Lake County,Antioch,L,Well 7 TP,GW,EPTDS from Well 7,2014-06-03,794714-11552,PFHxS,0.030,...,5,IL,2014,6,1,0,0.0,78001.0,12.6,9.5
655075,Adams County,ADAMS CO W/A #-KAISER LAKE,S,ADAMS CO W/A #4-KAISER LAKE,GW,Treatment Plant T101,2024-01-30,112089P,PFMPA,0.004,...,4,MS,2024,1,1,0,0.0,NaN,NaN,NaN
959859,Dauphin County,MILLERSBURG WATER AUTH,S,TREATMENT PLANT 301,SW,Entry Point 301,2023-02-14,101191P,PFMBA,0.003,...,3,PA,2023,2,1,0,0.0,73929.0,19.0,12.9
447837,Will County,FRANKFORT,L,TP 3 (Well 4),GW,TP 3 Finished Water,2023-01-25,TP03,PFBS,0.003,...,5,IL,2023,1,1,0,0.0,103587.0,9.3,7.2


In [31]:
# Missing values
df_PFAS_missing = df_PFAS.isna().sum()
print(df_PFAS_missing)


County                            6234
PWSName                              0
Size                                 0
FacilityName                         0
FacilityWaterType                    0
SamplePointName                      0
CollectionDate                       0
SampleID                             0
Contaminant                          0
MRL                                  0
AnalyticalResultsSign                0
AnalyticalResultValue                0
Region                               0
State                                0
Year                                 0
Month                                0
Count_Less_Than                      0
Count_Equals                         0
Relative_Contamination_Level         0
Median_Income                   712413
Child_Poverty_Rate              712413
Poverty_Rate                    712413
dtype: int64


In [32]:
'''Splitting up dataframe for export and upload to Github'''

#Split into 12 separate files by rows

#Set split size
split_size=len(df_PFAS)//12

#Split big_df
df1 = df_PFAS.iloc[:split_size]
df2 = df_PFAS.iloc[split_size:2*split_size]
df3 = df_PFAS.iloc[2*split_size:3*split_size]
df4 = df_PFAS.iloc[3*split_size:4*split_size]
df5 = df_PFAS.iloc[4*split_size:5*split_size]
df6 = df_PFAS.iloc[5*split_size:6*split_size]
df7 = df_PFAS.iloc[6*split_size:7*split_size]
df8 = df_PFAS.iloc[7*split_size:8*split_size]
df9 = df_PFAS.iloc[8*split_size:9*split_size]
df10 = df_PFAS.iloc[9*split_size:10*split_size]
df11 = df_PFAS.iloc[10*split_size:11*split_size]
df12 = df_PFAS.iloc[11*split_size:]

#Array of segmented dfs
dfs=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]

#Export to csv
for i, file in enumerate(dfs,1):
    file.to_csv(f'df{i}.csv',index=False)